<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml1027/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import zipfile
path = "/content/drive/MyDrive/additional/train.zip"
f = zipfile.ZipFile(path)
f.extractall()

In [ ]:
import glob
import matplotlib.pyplot as plt
from PIL import Image
fs = glob.glob("train/*")
plt.imshow(Image.open(fs[10000]))

In [15]:
import random
import pandas as pd
dogs = glob.glob("train/dog.*")
random.shuffle(dogs)
cats = glob.glob("train/cat.*")
random.shuffle(cats)
train = pd.DataFrame({
    "path":dogs[:900] + cats[:900],
    "target":[0] * 900 + [1] * 900
})
validate = pd.DataFrame({
    "path":dogs[900:1000] + cats[900:1000],
    "target":[0] * 100 + [1] * 100
})

In [17]:
validate

,path,target
0,train/dog.7410.jpg,0
1,train/dog.4745.jpg,0
2,train/dog.1208.jpg,0
3,train/dog.3307.jpg,0
4,train/dog.4097.jpg,0
...,...,...
195,train/cat.11629.jpg,1
196,train/cat.8217.jpg,1
197,train/cat.8486.jpg,1
198,train/cat.2631.jpg,1
